In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as tutils

In [3]:
batch_size = 100
imgDim = 28
path = './genImgConv/'

In [4]:
# MNIST dataset
dataset = datasets.MNIST(root='../Data/MNIST',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)


In [5]:
IS_CUDA = False
if torch.cuda.is_available():
    IS_CUDA = True
    
def var(x):
    if IS_CUDA:
        x = x.cuda()
    return Variable(x)

def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


In [7]:
atch_size = 128
lr = 0.0002
train_epoch = 20

# data_loader
img_size = 64

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

# G(z)
class generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 1, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = F.relu(self.deconv1_bn(self.deconv1(input)))
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv5(x))

        return x

class discriminator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d*8)
        self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv5(x))

        return x
    
G = generator(128)
D = discriminator(128)
G.weight_init(mean=0.0, std=0.02)
D.weight_init(mean=0.0, std=0.02)

if IS_CUDA:
    D.cuda()
    G.cuda()

In [5]:
'''
Generator_input = 64

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = x.resize(batch_size, 1, imgDim, imgDim)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = self.conv2(x)
        x = F.relu(F.max_pool2d(self.conv2_drop(x),2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

Generator_input = 64
Generator = nn.Sequential(
        nn.Linear(Generator_input, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 784),
        nn.Tanh())

D = Discriminator()

if IS_CUDA:
    D.cuda()
    Generator.cuda()
'''

In [9]:
lossCriterion = nn.BCELoss()
G_opt = torch.optim.Adam(G.parameters(), lr = 0.0001)
D_opt = torch.optim.Adam(D.parameters(), lr = 0.0001)
fixed_x = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)    # fixed noise
fixed_x = Variable(fixed_z_.cuda(), volatile=True)
#fixed_x = var(torch.randn(batch_size, Generator_input))

NameError: name 'Generator_input' is not defined

In [7]:
outputImages = []
def train(num_epochs = 10, d_iter = 1):
    for epoch in range(num_epochs):
        for data in data_loader:
            image, _  = data
            image = var(image.view(image.size(0),  -1))
            # Train Discriminator
            #for k in range(0, d_iter):
            # For Log D(x)
            #print image.shape
            D_real = D(image)
            # For Log(1 - D(G(Z)))
            Z_noise = var(torch.randn(batch_size, Generator_input))
            G_fake = Generator(Z_noise)
            D_fake = D(G_fake)

            # Calculate Discriminator Loss
            D_real_loss = lossCriterion(D_real, Variable(torch.ones(batch_size, 1)))
            D_fake_loss = lossCriterion(D_fake, Variable(torch.zeros(batch_size, 1)))
            D_loss = D_real_loss + D_fake_loss

            # Backprop Discriminator
            D.zero_grad()
            D_loss.backward()
            D_opt.step()

                
            # Train Generator
            Z_noise = var(torch.randn(batch_size, Generator_input))
            G_fake = Generator(Z_noise)
            D_fake = D(G_fake)
            # Compute Generator Loss
            G_loss = lossCriterion(D_fake, Variable(torch.ones(batch_size, 1)))
            
            # Backprop Genearator
            D.zero_grad()
            Generator.zero_grad()
            G_loss.backward()
            G_opt.step()
            
        #print epoch
        print 'Epoch [{}/{}], Discriminator {:.4f}, Generator {:.4f}'.format(epoch+1, num_epochs, D_loss.data[0], G_loss.data[0])
        pic = Generator(fixed_x)
        pic = pic.view(pic.size(0), 1, imgDim, imgDim) 
        outputImages.append(pic)
        torchvision.utils.save_image(pic.data, path+'image_{}.png'.format(epoch))             

In [11]:
train(30)

Epoch [1/30], Discriminator 0.0863, Generator 4.4198
Epoch [2/30], Discriminator 0.1273, Generator 4.2157
Epoch [3/30], Discriminator 0.0876, Generator 3.9506
Epoch [4/30], Discriminator 0.1807, Generator 4.4667
Epoch [5/30], Discriminator 0.1471, Generator 4.7031
Epoch [6/30], Discriminator 0.1103, Generator 4.8009
Epoch [7/30], Discriminator 0.0477, Generator 5.0006
Epoch [8/30], Discriminator 0.0580, Generator 4.6146
Epoch [9/30], Discriminator 0.0382, Generator 4.8695
Epoch [10/30], Discriminator 0.0297, Generator 4.7791
Epoch [11/30], Discriminator 0.0409, Generator 4.6476
Epoch [12/30], Discriminator 0.0260, Generator 5.5762
Epoch [13/30], Discriminator 0.0327, Generator 5.2278
Epoch [14/30], Discriminator 0.0582, Generator 5.6335
Epoch [15/30], Discriminator 0.0403, Generator 5.6577
Epoch [16/30], Discriminator 0.0278, Generator 4.9055
Epoch [17/30], Discriminator 0.0228, Generator 5.4630
Epoch [18/30], Discriminator 0.0852, Generator 5.4339
Epoch [19/30], Discriminator 0.0221, 

In [9]:
#torch.save(Generator.state_dict(), './model.load_state_dict(torch.load('mytraining.pt')).pkl')
#torch.save(Discriminator.state_dict(), './Discriminator.pkl')

SyntaxError: invalid syntax (<ipython-input-9-70e24908820e>, line 1)

In [ ]:
#Generator.load_state_dict(torch.load('Generator200.pkl'))
#Discriminator.load_state_dict(torch.load('Discriminator200.pkl'))